### Домашнее задание 3:

Используя файл Lesson_3_extended.ipynb (он в web3.zip в материалах):

* Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

* Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.
Могла ли модель переобучиться? Почему?

In [69]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d.axes3d import Axes3D

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')



Задача: предсказание баллов ЕГЭ ученика

In [70]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)



In [71]:
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

Стандартизация

In [72]:
def standard_scale(X):
    mean = X.mean(axis=0)
    std = X.std(axis=0)
    return (X - mean) / std

In [73]:
X_st = X.copy().astype(np.float64)
X_st[:, 1] = standard_scale(X_st[:, 1])
X_st[:, 2] = standard_scale(X_st[:, 2])
X_st[:, 3] = standard_scale(X_st[:, 3])

X_st

array([[ 1.        , -0.70710678, -0.97958969, -0.89625816],
       [ 1.        , -0.70710678, -0.56713087, -0.89625816],
       [ 1.        , -0.35355339, -0.46401617,  0.38411064],
       [ 1.        ,  0.70710678, -0.77336028, -0.89625816],
       [ 1.        ,  0.        ,  0.97958969,  0.38411064],
       [ 1.        , -1.06066017, -0.36090146, -0.89625816],
       [ 1.        ,  0.70710678,  1.08270439,  1.66447944],
       [ 1.        ,  2.47487373,  2.11385144,  1.66447944],
       [ 1.        , -0.70710678, -1.08270439, -0.89625816],
       [ 1.        , -0.35355339,  0.05155735,  0.38411064]])

### 1. * Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.


In [74]:
def calc_logloss(y, y_pred):
    y_pred = np.where(y_pred == 0, 0 + 1e-6, y_pred)
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

### 2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.


Logistic Regression

In [75]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [76]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [77]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [78]:
W = eval_model(X_st, y, iterations=500, eta=10)

0 [ 0.14172249 -2.84645945 -1.68657857  0.27823209] 0.760958797591889
50 [ 1.16099815 -6.53918109 -4.4949357  11.04277706] 0.27763870943571517
100 [ 2.03780151 -7.67437695 -6.65766302 14.86839912] 0.22027159779808106
150 [ 2.61747961 -8.40748156 -7.81457011 17.68619185] 0.16005020313927093
200 [ 2.96203929 -8.94776961 -8.55425479 19.78680905] 0.12824405634491978
250 [ 3.17054497 -9.41689153 -9.13965373 21.46758381] 0.11515140267334115
300 [ 3.37974227 -9.9389475  -9.74543673 22.85773885] 0.10978159192651307
350 [  3.57947845 -10.42730717 -10.31253627  24.10523393] 0.1054546743475953
400 [  3.76169379 -10.87310575 -10.83732084  25.25364704] 0.10179195658323752
450 [  3.92986323 -11.28378466 -11.32845215  26.32214686] 0.0986246100987463


### 3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [79]:
def calc_pred_proba(W, X):
    y_pred_proba = sigmoid(np.dot(X, W))
    return y_pred_proba

In [80]:
calc_pred_proba(W, X_st)

array([3.53618676e-01, 4.22181553e-03, 1.00000000e+00, 3.33167074e-09,
       9.53873107e-01, 2.24961201e-02, 1.00000000e+00, 1.43917443e-02,
       6.48363091e-01, 9.99999986e-01])

### 4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [81]:
def calc_pred(W, X):
    y_pred_proba = calc_pred_proba(W, X)
    y_pred = np.where(y_pred_proba > 0.5, 1, 0)
    return y_pred

In [82]:
calc_pred(W, X_st)

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])